In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [121]:
Qinfo = pd.read_csv('data/QInfo.csv')
Qinfo.head()

,Unnamed: 0,QID,title,titleSlug,difficulty,acceptanceRate,paidOnly,topicTags,categorySlug
0,0,1,Two Sum,two-sum,Easy,50.988683,False,"array,hash-table",algorithms
1,1,2,Add Two Numbers,add-two-numbers,Medium,41.483527,False,"linked-list,math,recursion",algorithms
2,2,3,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,Medium,34.113931,False,"hash-table,string,sliding-window",algorithms
3,3,4,Median of Two Sorted Arrays,median-of-two-sorted-arrays,Hard,38.292365,False,"array,binary-search,divide-and-conquer",algorithms
4,4,5,Longest Palindromic Substring,longest-palindromic-substring,Medium,33.227066,False,"string,dynamic-programming",algorithms


In [3]:
Qbody = pd.read_csv('data/Qbody.csv')
Qbody

,Unnamed: 0,QID,Hints,Companies,SimilarQuestions,Code,Body,isPaidOnly
0,0,1,"[""A really brute force way would be to search ...",NaN,"[15, 18, 167, 170, 560, 653, 1099, 1679, 1711,...","class Solution:\n def twoSum(self, nums: Li...",<p>Given an array of integers <code>nums</code...,False
1,1,2,[],NaN,"[43, 67, 371, 415, 445, 989, 1634, 2816]",# Definition for singly-linked list.\n# class ...,<p>You are given two <strong>non-empty</strong...,False
2,2,3,[],NaN,"[159, 340, 992, 1695, 2067, 2260, 2401, 2405, ...",class Solution:\n def lengthOfLongestSubstr...,"<p>Given a string <code>s</code>, find the len...",False
3,3,4,[],NaN,[2387],class Solution:\n def findMedianSortedArray...,<p>Given two sorted arrays <code>nums1</code> ...,False
4,4,5,['How can we reuse a previously computed palin...,NaN,"[214, 266, 336, 516, 647, 2472]",class Solution:\n def longestPalindrome(sel...,"<p>Given a string <code>s</code>, return <em>t...",False
...,...,...,...,...,...,...,...,...
2337,2337,2916,['Consider the sum of the count of distinct va...,NaN,[],"class Solution:\n def sumCounts(self, nums:...",<p>You are given a <strong>0-indexed </strong>...,False
2338,2338,2917,['Fix a <code>bit</code> from the range <code>...,NaN,"[338, 2859]","class Solution:\n def findKOr(self, nums: L...",<p>You are given a <strong>0-indexed</strong> ...,False
2339,2339,2918,['Consider we replace all the 0’s with 1’s on ...,NaN,[],"class Solution:\n def minSum(self, nums1: L...",<p>You are given two arrays <code>nums1</code>...,False
2340,2340,2919,['There needs to be at least one value among <...,NaN,[],class Solution:\n def minIncrementOperation...,<p>You are given a <strong>0-indexed</strong> ...,False


In [4]:
Qbody.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2342 non-null   int64  
 1   QID               2342 non-null   int64  
 2   Hints             2342 non-null   object 
 3   Companies         0 non-null      float64
 4   SimilarQuestions  2342 non-null   object 
 5   Code              2342 non-null   object 
 6   Body              2342 non-null   object 
 7   isPaidOnly        2342 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 130.5+ KB


In [5]:
sample_html = Qbody['Body'][442]
sample_html

'Given a list of 24-hour clock time points in <strong>&quot;HH:MM&quot;</strong> format, return <em>the minimum <b>minutes</b> difference between any two time-points in the list</em>.\n<p>&nbsp;</p>\n<p><strong class="example">Example 1:</strong></p>\n<pre><strong>Input:</strong> timePoints = ["23:59","00:00"]\n<strong>Output:</strong> 1\n</pre><p><strong class="example">Example 2:</strong></p>\n<pre><strong>Input:</strong> timePoints = ["00:00","23:59","00:00"]\n<strong>Output:</strong> 0\n</pre>\n<p>&nbsp;</p>\n<p><strong>Constraints:</strong></p>\n\n<ul>\n\t<li><code>2 &lt;= timePoints.length &lt;= 2 * 10<sup>4</sup></code></li>\n\t<li><code>timePoints[i]</code> is in the format <strong>&quot;HH:MM&quot;</strong>.</li>\n</ul>\n'

In [6]:
soup = BeautifulSoup(sample_html, 'html.parser')
print(soup.text)

Given a list of 24-hour clock time points in "HH:MM" format, return the minimum minutes difference between any two time-points in the list.
 
Example 1:
Input: timePoints = ["23:59","00:00"]
Output: 1
Example 2:
Input: timePoints = ["00:00","23:59","00:00"]
Output: 0

 
Constraints:

2 <= timePoints.length <= 2 * 104
timePoints[i] is in the format "HH:MM".




In [59]:
def parse_question(question_html):
    soup = BeautifulSoup(question_html, 'html.parser')
    txt = soup.text.replace('\xa0', ' ').split('\n \n')[0]
    #match = re.search(r'(.+?\n\n)', txt, re.DOTALL)
    #txt = match.group(0)
    txt = txt.replace('-', ' ')
    txt = txt.replace('\n', ' ')
    return re.sub(r'[^\w\s]', '', txt).rstrip().lower()
parse_question(Qbody['Body'][0])

'given an array of integers nums and an integer target return indices of the two numbers such that they add up to target you may assume that each input would have exactly one solution and you may not use the same element twice you can return the answer in any order'

In [60]:
Qbody['text'] = Qbody['Body'].apply(parse_question)

In [61]:
vocab = set()
def update_set(txt):
    vocab.update(txt.split())
Qbody['text'].apply(update_set)
len(vocab)

7534

In [62]:
list(vocab)[:10]

['displays',
 'let',
 'free',
 'operates',
 'uniquely',
 'vote',
 'x1x2x3x4',
 'worry',
 'factory',
 'sqrt']

In [63]:
df = pd.DataFrame(
    {
        'word' : list(vocab)
    }
)
df.head()

,word
0,displays
1,let
2,free
3,operates
4,uniquely


In [64]:
def term_freq_fnc(doc):
    def term_freq(word):
        doc_words = doc.split()
        return (doc_words.count(word)/len(doc_words))
    return term_freq

In [65]:
term_freq = term_freq_fnc(Qbody['text'][0])
tmp_df = df['word'].apply(term_freq)

In [66]:
tmp_df[tmp_df>0][:10]

22     0.019608
37     0.019608
148    0.019608
294    0.019608
388    0.019608
406    0.019608
440    0.019608
687    0.019608
701    0.019608
755    0.019608
Name: word, dtype: float64

In [67]:
for i, doc in enumerate(Qbody['text']):
    term_freq = term_freq_fnc(doc)
    df[f'TF{i+1}'] = df['word'].apply(term_freq)

C:\Users\heyse\AppData\Local\Temp\ipykernel_9416\3224010180.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'TF{i+1}'] = df['word'].apply(term_freq)
C:\Users\heyse\AppData\Local\Temp\ipykernel_9416\3224010180.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'TF{i+1}'] = df['word'].apply(term_freq)
C:\Users\heyse\AppData\Local\Temp\ipykernel_9416\3224010180.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

In [68]:
no_of_docs = len(Qbody['text'])
no_of_docs

2342

In [69]:
tf_cols = [f'TF{i+1}' for i in range(no_of_docs)]
doc_freq = np.sum( (df[tf_cols] > 0).to_numpy(), axis=1, keepdims=True)
doc_freq

array([[ 2],
       [46],
       [12],
       ...,
       [ 1],
       [ 2],
       [ 2]])

In [70]:
idf = np.log(no_of_docs / doc_freq)
idf

array([[7.06561336],
       [3.93011915],
       [5.27385389],
       ...,
       [7.75876054],
       [7.06561336],
       [7.06561336]])

In [71]:
tf = df[tf_cols].to_numpy()
tf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
tf_idf = tf * idf

In [73]:
tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [74]:
tmp = tf_idf.sum(axis=1)
np.where(tmp <= 0)

(array([], dtype=int64),)

In [75]:
df

,word,TF1,TF2,TF3,TF4,TF5,TF6,TF7,TF8,TF9,...,TF2333,TF2334,TF2335,TF2336,TF2337,TF2338,TF2339,TF2340,TF2341,TF2342
0,displays,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,let,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.011905,0.0,0.0,0.010417,0.0,0.0,0.0,0.0
2,free,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,operates,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,uniquely,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7529,builddictstring,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7530,lengthi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7531,2932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7532,subordinates,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [76]:
tfidf_DF = pd.DataFrame(
    tf_idf,
    columns=[f'TF_IDF{i+1}' for i in range(no_of_docs)]
)
tfidf_DF['word'] = df['word']
tfidf_DF

,TF_IDF1,TF_IDF2,TF_IDF3,TF_IDF4,TF_IDF5,TF_IDF6,TF_IDF7,TF_IDF8,TF_IDF9,TF_IDF10,...,TF_IDF2334,TF_IDF2335,TF_IDF2336,TF_IDF2337,TF_IDF2338,TF_IDF2339,TF_IDF2340,TF_IDF2341,TF_IDF2342,word
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,displays
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.046787,0.0,0.0,0.040939,0.0,0.0,0.0,0.0,let
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,free
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,operates
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,uniquely
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,builddictstring
7530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,lengthi
7531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2932
7532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,subordinates


In [77]:
tfidf_DF.iloc[438]

TF_IDF1              0.0
TF_IDF2              0.0
TF_IDF3              0.0
TF_IDF4              0.0
TF_IDF5              0.0
                 ...    
TF_IDF2339           0.0
TF_IDF2340           0.0
TF_IDF2341           0.0
TF_IDF2342           0.0
word          constructs
Name: 438, Length: 2343, dtype: object

In [78]:
tfidf_DF

,TF_IDF1,TF_IDF2,TF_IDF3,TF_IDF4,TF_IDF5,TF_IDF6,TF_IDF7,TF_IDF8,TF_IDF9,TF_IDF10,...,TF_IDF2334,TF_IDF2335,TF_IDF2336,TF_IDF2337,TF_IDF2338,TF_IDF2339,TF_IDF2340,TF_IDF2341,TF_IDF2342,word
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,displays
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.046787,0.0,0.0,0.040939,0.0,0.0,0.0,0.0,let
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,free
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,operates
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,uniquely
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,builddictstring
7530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,lengthi
7531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2932
7532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,subordinates


In [93]:
import numpy as np

# Create a sample 2D NumPy array
array_2d = np.array([
    [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5, 8, 9, 7, 9, 3, 2, 3, 8, 4],
    [4, 2, 5, 7, 3, 1, 8, 3, 5, 6, 1, 9, 4, 2, 7, 8, 6, 1, 3, 5]
])

# Find the positions of the highest 20 values in each row
k = 2
highest_positions = []
for row in array_2d:
    partitioned_indices = np.argpartition(row, -k)[-k:]
    sorted_indices = partitioned_indices[np.argsort(row[partitioned_indices])]
    highest_positions.append(sorted_indices)

print(f"Positions of the highest {k} values in each row:")
for i, positions in enumerate(highest_positions):
    print(f"Row {i}: {positions}", [array_2d[i, p] for p in positions])
np.array(highest_positions)

Positions of the highest 2 values in each row:
Row 0: [14  5] [9, 9]
Row 1: [ 6 11] [8, 9]


array([[14,  5],
       [ 6, 11]], dtype=int64)

In [100]:
k = 20
highest_positions = []
for row in tf_idf:
    partitioned_indices = np.argpartition(row, -k)[-k:]
    sorted_indices = partitioned_indices[np.argsort(row[partitioned_indices])]
    highest_positions.append(sorted_indices)
highest_positions = np.array(highest_positions)

In [101]:
highest_positions

array([[ 769,  772,  773, ...,  771,  604,  852],
       [1943,  962, 2188, ..., 1525, 2094, 1995],
       [ 779,  769,  770, ..., 1521, 2072, 2012],
       ...,
       [ 788,  772,  773, ...,  771, 2341, 1739],
       [ 787,  772,  773, ...,  780, 1116,  554],
       [ 770,  773,  774, ..., 2341,  353, 1623]], dtype=int64)

In [109]:
f"{(highest_positions.size * highest_positions.itemsize)//(1024)} kb"

'1177 kb'

In [110]:
lookup_table = pd.DataFrame(
    highest_positions,
    columns=[f'DOC {i}' for i in range(20)]
)
lookup_table['word'] = df['word']
lookup_table

,DOC 0,DOC 1,DOC 2,DOC 3,DOC 4,DOC 5,DOC 6,DOC 7,DOC 8,DOC 9,...,DOC 11,DOC 12,DOC 13,DOC 14,DOC 15,DOC 16,DOC 17,DOC 18,DOC 19,word
0,769,772,773,774,775,776,777,778,779,2341,...,781,782,783,784,785,770,771,604,852,displays
1,1943,962,2188,2334,2277,1097,2164,2184,2087,1468,...,638,2322,2065,2221,592,1474,1525,2094,1995,let
2,779,769,770,771,778,773,776,772,1296,2037,...,1066,618,1021,1581,2210,1727,1521,2072,2012,free
3,769,772,773,774,775,776,777,771,778,780,...,782,783,784,785,770,779,2341,810,1761,operates
4,801,773,774,775,776,777,778,772,779,781,...,783,784,771,780,2341,1689,815,1509,1952,uniquely
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7529,788,772,773,774,775,776,777,778,771,779,...,782,783,784,770,786,803,780,2341,542,builddictstring
7530,797,773,774,775,776,777,778,779,780,781,...,783,784,785,771,803,788,772,2341,1370,lengthi
7531,788,772,773,774,775,776,777,778,779,780,...,782,783,784,785,770,769,771,2341,1739,2932
7532,787,772,773,774,775,776,777,778,771,779,...,782,783,784,770,785,769,780,1116,554,subordinates


In [123]:
searh_word = "node"

idx = np.where((lookup_table['word'] == searh_word).to_numpy())[0]
if idx.size == 0:
    print('Not found')
else:
    idx = idx[0]
    doc_id = lookup_table['DOC 19'][idx]
    print(Qbody['text'][doc_id])
    print(Qinfo.iloc[doc_id])

given a binary tree  struct node    int val   node left   node right   node next   populate each next pointer to point to its next right node if there is no next right node the next pointer should be set to null initially all next pointers are set to null
Unnamed: 0                                                      116
QID                                                             117
title                Populating Next Right Pointers in Each Node II
titleSlug            populating-next-right-pointers-in-each-node-ii
difficulty                                                   Medium
acceptanceRate                                            51.402016
paidOnly                                                      False
topicTags         linked-list,tree,depth-first-search,breadth-fi...
categorySlug                                             algorithms
Name: 116, dtype: object


In [126]:
# lookup table and Qinfo would be used by the app
Qinfo.to_csv('QINFO.csv', index=False, columns=Qinfo.columns[1:])
lookup_table.to_csv('LOOKUP.csv', index=False)